In [1]:
%%capture

%pip install langchain-community -U
%pip install langchain-google-genai
%pip install pypdf
%pip install chromadb
%pip install langchain
%pip install langchain-chroma
%pip install langchain -U

In [2]:
# Imports
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import chromadb
import uuid
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_chroma import Chroma
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [4]:
# Carregar e extrair pdf
loader = PyPDFLoader('os-sertoes.pdf')
documents = loader.load()

# Criar chuncks
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=20)
texts = text_splitter.split_documents(documents)

In [5]:
# Embeddings
os.environ['GOOGLE_API_KEY'] = os.getenv('GEMINI_API_KEY')

embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
# Extrair o conteúdo de cada Document para gerar embeddings
text_contents = [doc.page_content for doc in texts]
embeddings = embeddings_model.embed_documents(text_contents)

In [6]:
# VectorDB
client = chromadb.PersistentClient()
collection = client.get_or_create_collection(name="pdf_documents")

# Gerar IDs únicos para cada chunk e adicionar ao ChromaDB
ids = [str(uuid.uuid4()) for _ in range(len(texts))]
collection.add(
    embeddings=embeddings,
    documents=text_contents,
    ids=ids
)


In [7]:
# retriever 
vectorstore = Chroma(
    client=client,
    collection_name="pdf_documents",
    embedding_function=embeddings_model
)

# 5. Transforma a coleção em um retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [8]:
# Inicialize o modelo Gemini
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2)
# Crie o template do prompt
prompt = ChatPromptTemplate.from_template("""
    Você é um bibliotecário. Responda as perguntas baseadas no contexto fornecido.
                                          
    Context: {context}
                                          
    Pergunta: {input}
""")

document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [10]:
asks = [
    "Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?",
    "Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?",
    "Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?",
    "Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?",
    "Quais são os principais aspectos da crítica social e política presentes em \"Os Sertões\"? Como esses aspectos refletem a visão do autor sobre o Brasil da época?"
]

for ask in asks:
    # Chame a Chain com uma pergunta
    response = retrieval_chain.invoke({"input": ask})

    # Imprima a resposta
    print(f"Pergunta: {ask}\nResposta: {response['answer']}\n\n")

Pergunta: Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?
Resposta: Euclides da Cunha descreve o sertão nordestino como um ambiente hostil, marcado pela seca e pela aridez.  Ele vê a falta de água como o problema central, argumentando que a ausência de um "mar interior" que pudesse moderar o clima,  torna a região um "deserto".  Soluções pontuais como cisternas e poços artesianos apenas amenizam a sede, mas não resolvem o problema fundamental.

Para Cunha, o sofrimento humano ("martírio do homem") é um reflexo direto do sofrimento da terra ("martírio secular da Terra"). A natureza inclemente impõe condições de vida extremamente difíceis, afetando profundamente a economia e a própria sobrevivência da população.  A falta de estudos científicos aprofundados sobre a região, demonstrados pela passagem rápida de pesquisadores como Martius, reforça a ideia de um ambiente pouco compreendido e, consequentemente, mal 